A factory produces three types of beds, A, B, and C. The company that owns the factory sells beds of type A for \\$ 250 each, beds of type B for \\$320.00 each, and beds of type C for \\$625 each. Management estimates that all beds produced of types A and C will be sold. The number of beds of type B that can be sold is at the most 45. The production of diﬀerent types of beds require s a diﬀerent amount of resources such as basic labor hours, specialized hours, and materials. The following table provides these data<br>

|Type A| Type B |Type C|Resource|
|:-|:-|:-|:-|
|10ft|60 ft|80 ft|Material|
15 min|20Min|40 min|basic labor|
5 min|15 min|20 min|Specialized labor

The amounts of resources available are 450 ft of material, 210 hoursof basic labor, and 95 hours of specialized labor. The problem is to optimize sales. and solve using pulp.

Source: Introduction to Computational Models with Python,Chapter 22

In [1]:
from pulp import *

In [88]:
#Number of variables, range of the variables
numBeds=3
rBeds=range(1,numBeds+1)

In [89]:
#revenue vector
rev=[250,320,625]
bedRev={rBeds[i]:rev[i] for i in range(numBeds)}

In [90]:
bedRev

{1: 250, 2: 320, 3: 625}

In [91]:
## Resource Available
numResource=3
Resource=range(1,numResource+1)
resAvail = [[10,15,5],
            [60,20,15],
            [80,40,20]   
]
#variable1:resource1
#Resource 1 coeff in roww 1
bedResAvail={(i,j):resAvail[i-1][j-1] for j in range(1,numResource+1) for i in range(1,numBeds+1)}

In [92]:
bedResAvail

{(1, 1): 10,
 (2, 1): 60,
 (3, 1): 80,
 (1, 2): 15,
 (2, 2): 20,
 (3, 2): 40,
 (1, 3): 5,
 (2, 3): 15,
 (3, 3): 20}

In [93]:
upbound=[450,210,95]
upbound1={i+1:upbound[i] for i in range(numResource)}

In [94]:
upbound1# 1=Material,2=Labor, 3=Special labor

{1: 450, 2: 210, 3: 95}

#### Building the LP model in PuLP

In [118]:
model=LpProblem("Problem.2",LpMaximize)

#### Decision Variables

In [119]:
decVar=LpVariable.dicts(name="x",indexs=rBeds,lowBound=0,upBound=None,cat="Integer")

#### Objective function

In [120]:
lpSum([bedRev[i]*decVar[i] for i in range(1,numBeds+1)])

250*x_1 + 320*x_2 + 625*x_3 + 0

In [121]:
model += lpSum([bedRev[i]*decVar[i] for i in range(1,numBeds+1)]),"objFunction"

#### Constraints

In [122]:
#### Resource cconstraints
c={1:"Material.Resource",2:"Basic.Labor",3:"Special.Labor"}
for r in Resource:
    print(lpSum([bedResAvail[j,r]*decVar[j] for j in rBeds])<=upbound1[r],"%s"%c[r])

model += lpSum([bedResAvail[j,r]*decVar[j] for j in rBeds])<=upbound1[r],"%s"%c[r]

10*x_1 + 60*x_2 + 80*x_3 <= 450 Material.Resource
15*x_1 + 20*x_2 + 40*x_3 <= 210 Basic.Labor
5*x_1 + 15*x_2 + 20*x_3 <= 95 Special.Labor


In [123]:
### Bed B can be sold atmost 45
model+=decVar[2]<=45

#### Solving the model|

In [124]:
model.solve(GLPK())

1

In [125]:
print("Problem Status:", LpStatus[model.status])

Problem Status: Optimal


In [126]:

for v in model.variables():
    print(v.name, "=", v.varValue)

x_1 = 19
x_2 = 0
x_3 = 0


In [130]:
#Bed A sold= 19

In [131]:
print("The total sales of beds: ", value(model.objective))

The total sales of beds:  4750


625